In [ ]:
import random
import math

# ==========================================
# 0. EL MOTOR MATEMÁTICO (REEMPLAZANDO NUMPY)
# ==========================================

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Función para aplicar Sigmoide a una matriz completa (lista de listas)
def apply_function(matrix, func):
    return [[func(x) for x in row] for row in matrix]

# Multiplicación de Matrices (Dot Product)
# Esta es la operación más costosa que NumPy hace en C
def matmul(A, B):
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
        raise ValueError("Dimensiones incompatibles para multiplicación")

    # Crear matriz resultado llena de ceros
    result = [[0 for _ in range(cols_B)] for _ in range(rows_A)]

    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]
    return result

# Transponer una matriz (Intercambiar filas por columnas)
def transpose(A):
    return [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]

# Operaciones Elemento a Elemento (Suma, Resta, Multiplicación)
def matrix_sub(A, B):
    return [[A[i][j] - B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

def matrix_mul_scalar(A, scalar):
    return [[A[i][j] * scalar for j in range(len(A[0]))] for i in range(len(A))]

def matrix_multiply_elementwise(A, B):
    return [[A[i][j] * B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

def matrix_add(A, B):
    return [[A[i][j] + B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

# Suma para los Sesgos (Biases) - Sumar columnas
def sum_columns(A):
    # Retorna una lista de listas [[sum1, sum2...]]
    sums = [0] * len(A[0])
    for row in A:
        for i, val in enumerate(row):
            sums[i] += val
    return [sums] # Retornamos como matriz 1xN

# ==========================================
# 1. PREPARACIÓN DE DATOS
# ==========================================
random.seed(42)

# Inputs (X): 4 filas, 2 columnas
X = [[0, 0], [0, 1], [1, 0], [1, 1]]

# Targets (y): 4 filas, 1 columna
y = [[0], [1], [1], [0]]

# ==========================================
# 2. INICIALIZACIÓN
# ==========================================

input_neurons = 2
hidden_neurons = 2
output_neurons = 1

# Inicializar pesos aleatorios entre 0 y 1
# W1: 2x2
weights_input_hidden = [[random.uniform(0, 1) for _ in range(hidden_neurons)] for _ in range(input_neurons)]
# B1: 1x2
bias_hidden = [[random.uniform(0, 1) for _ in range(hidden_neurons)]]
# W2: 2x1
weights_hidden_output = [[random.uniform(0, 1) for _ in range(output_neurons)] for _ in range(hidden_neurons)]
# B2: 1x1
bias_output = [[random.uniform(0, 1) for _ in range(output_neurons)]]

lr = 0.5
epochs = 10000

print("Entrenando Red Neuronal en Python Puro...")

# ==========================================
# 3. BUCLE DE ENTRENAMIENTO
# ==========================================
for epoch in range(epochs):
    
    # --- FORWARD PASS ---
    # 1. Capa Oculta
    # Z_h = X . W1
    hidden_input = matmul(X, weights_input_hidden)
    # Z_h = Z_h + b1 (Broadcasting manual: sumar b1 a cada fila)
    for i in range(len(hidden_input)):
        for j in range(len(hidden_input[0])):
            hidden_input[i][j] += bias_hidden[0][j]

    # A_h = sigmoid(Z_h)
    hidden_output = apply_function(hidden_input, sigmoid)
    
    # 2. Capa de Salida
    # Z_o = A_h . W2
    final_input = matmul(hidden_output, weights_hidden_output)
    # Z_o = Z_o + b2
    for i in range(len(final_input)):
        for j in range(len(final_input[0])):
            final_input[i][j] += bias_output[0][j]

    # Predicted = sigmoid(Z_o)
    predicted_output = apply_function(final_input, sigmoid)
    
    # --- BACKPROPAGATION --
    # 1. Calcular Error (y - prediction)
    error = matrix_sub(y, predicted_output)
    
    # 2. Gradiente Salida
    # d_out = error * sigmoid_derivative(prediction)
    d_predicted = apply_function(predicted_output, sigmoid_derivative)
    d_output = matrix_multiply_elementwise(error, d_predicted)
    
    # 3. Gradiente Oculto
    # error_hidden = d_output . W2_Transpuesta
    error_hidden_layer = matmul(d_output, transpose(weights_hidden_output))
    # d_hidden = error_hidden * sigmoid_derivative(hidden_output)
    d_hidden_activations = apply_function(hidden_output, sigmoid_derivative)
    d_hidden_layer = matrix_multiply_elementwise(error_hidden_layer, d_hidden_activations)
    
    # --- OPTIMIZACIÓN (SGD) ---
    # W2 += hidden_T . d_output * lr
    delta_w2 = matmul(transpose(hidden_output), d_output)
    delta_w2 = matrix_mul_scalar(delta_w2, lr)
    weights_hidden_output = matrix_add(weights_hidden_output, delta_w2)
    
    # b2 += sum(d_output) * lr
    delta_b2 = sum_columns(d_output)
    delta_b2 = matrix_mul_scalar(delta_b2, lr)
    bias_output = matrix_add(bias_output, delta_b2)

    # W1 += X_T . d_hidden * lr
    delta_w1 = matmul(transpose(X), d_hidden_layer)
    delta_w1 = matrix_mul_scalar(delta_w1, lr)
    weights_input_hidden = matrix_add(weights_input_hidden, delta_w1)

    # b1 += sum(d_hidden) * lr
    delta_b1 = sum_columns(d_hidden_layer)
    delta_b1 = matrix_mul_scalar(delta_b1, lr)
    bias_hidden = matrix_add(bias_hidden, delta_b1)

    if epoch % 1000 == 0:
        # Calcular Mean Squared Error simple para reporte
        total_loss = sum([e[0]**2 for e in error]) / len(error)
        print(f"Epoch {epoch}: Loss {total_loss:.5f}")

    